Import necessary packages
=============

In [1]:
# In case we want to make changes to qclib and reload
from importlib import reload

# Import basic plotting and number manipulation
import numpy as np
import matplotlib as plt

# This is really just for printing 
import pandas as pd

# Import basic pyQuil programs
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.api import WavefunctionSimulator

# To make an arbitrary state, need a module from Grove
from grove.alpha.arbitrary_state.arbitrary_state import create_arbitrary_state

# Also need quantum fourier transform from Grove
from grove.qft.fourier import qft

# Import personal quantum computing libraries
import LowLevelQCAlgorithms as llqca
import HighLevelQCAlgorithms as hlqca
import QCHelper as qch

# Beginning Test of pyquil and qvm

* Note that, in separate command windows, you need to run the commands `qvm -S` and `quilc -S`

In [2]:
program = Program(
    H(0),
    CNOT(0, 1),
)
print(program)

H 0
CNOT 0 1



In [3]:
wfn = WavefunctionSimulator().wavefunction(program)
print(wfn)

(0.7071067812+0j)|00> + (0.7071067812+0j)|11>


# Run Tests For Custom Library Functions

### Test out Flip function

In [5]:
# Define qubit registers
a = [0, 1, 2, 3, 4, 5]
b = [6, 7, 8, 9, 10]

# Generate some superposition of qubits, print result
p = Program(H(0), X(1), X(2), X(3), X(4), H(5), I(6), I(7), I(8), I(9), I(10))
result = WavefunctionSimulator().wavefunction(p)
print('The initial state is:', result, sep='\n')
print()

# Do Flip function on qubits, print result
p.inst(llqca.Flip(a, b))
result = WavefunctionSimulator().wavefunction(p)
print('When we apply Flip, it becomes:', result, sep='\n')

The initial state is:
(0.5+0j)|00000011110> + (0.5+0j)|00000011111> + (0.5+0j)|00000111110> + (0.5+0j)|00000111111>

When we apply Flip, it becomes:
(0.5+0j)|00000011110> + (0.5+0j)|00000111110> + (0.5+0j)|01111011111> + (0.5+0j)|11111111111>


#### Results of test
* First basis vector has 0 in $a_0$ place, so no change to b's
* Second basis vector has all 1's to $a_4$, so $b_0-b_3$ is flipped
* Third basis vector has all 1's to $a_5$, so $b_0-b_4$ is flipped

Looks like it's working correctly

### Test out AndTemp function

In [6]:
# Define qubit registers
a = [0, 1, 2, 3, 4, 5]
c = [6, 7, 8, 9]
b = [10]

# Generate some superposition of qubits, print result
p = Program(H(0), X(1), X(2), X(3), X(4), H(5), I(6), I(7), I(8), I(9), I(10))
result = WavefunctionSimulator().wavefunction(p)
print('The initial state is:', result, sep='\n')
print()

# Do AndTemp function on qubits, print result
p.inst(llqca.AndTemp(a, b, c))
result = WavefunctionSimulator().wavefunction(p)
print('When we apply AndTemp, it becomes:', result, sep='\n')

The initial state is:
(0.5+0j)|00000011110> + (0.5+0j)|00000011111> + (0.5+0j)|00000111110> + (0.5+0j)|00000111111>

When we apply AndTemp, it becomes:
(0.5+0j)|00000011110> + (0.5+0j)|00000011111> + (0.5+0j)|00000111110> + (0.5+0j)|10000111111>


#### Results of test
* Only last basis vector has all 1's in the a register, so b is flipped in that case
* Also, none of the qubits in the c register are modified
Thus, it is working properly. 

### Test out And function

In [7]:
# Define qubit registers
a = [0, 1, 2, 3, 4, 5]
b = [6]
t = [7]

# Create some superposition of qubits, print results
p = Program(H(0), X(1), X(2), X(3), X(4), H(5), I(6), I(7))
result = WavefunctionSimulator().wavefunction(p)
print('The initial state is:', result, sep='\n')
print()

# Apply the And function to the superposition, print results
p.inst(llqca.And(a, b, t))
result = WavefunctionSimulator().wavefunction(p)
print('When we apply And, it becomes:', result, sep='\n')

The initial state is:
(0.5+0j)|00011110> + (0.5+0j)|00011111> + (0.5+0j)|00111110> + (0.5+0j)|00111111>

When we apply And, it becomes:
(0.5+0j)|00011110> + (0.5+0j)|00011111> + (0.5+0j)|00111110> + (0.5+0j)|01111111>


#### Results of test
* Qubit b flips only when all the qubits in register a are 1.
* Temporary qubit t is unchanged by the operation.

## Test out arbitrary/controlled gate functions
* To begin these tests, we first need to have an arbitrary gate that we want to test out.
* This can be done by defining an $\alpha$, $\beta$, $\delta$, $\gamma$ as in Rieffel and Polak:

$$
\left(
\begin{matrix}
e^{\bf{i}(\delta + \alpha + \gamma)}\cos\beta & e^{\bf{i}(\delta + \alpha - \gamma)}\sin\beta \\
-e^{\bf{i}(\delta - \alpha + \gamma)}\sin\beta & e^{\bf{i}(\delta - \alpha - \gamma)}\cos\beta
\end{matrix}
\right)
$$

* This matrix represents the most general type of single-qubit transformation.
* Here we pick some arbitrary parameters, calculate what the resulting matrix should look like, generate some transformations using the library, then extract the matrix representing those transformations to test against our original calculation.

In [8]:
# Define the arbitrary transformation
Q = {'delta': np.pi/3, 'alpha': np.pi/5, 'beta': np.pi/4, 'gamma': np.pi/6}

# Calculate what the matrix representation of this transformation should look like
print(qch.arbitraryGate(Q))

[[-0.41562694+0.5720614j   0.28760624+0.64597419j]
 [-0.41562694-0.5720614j   0.70323318-0.07391279j]]


### Arbitrary Gate Function

In [9]:
# Define target qubit, gate transformation
t = [0]
Q = {'delta': np.pi/3, 'alpha': np.pi/5, 'beta': np.pi/4, 'gamma': np.pi/6}
n_qubits = 1

# Apply transformation to qubit, print result
p = Program()
p.inst( llqca.arbitraryGate(Q, t) )

# Get matrix representing transformation
qch.getMatFromProgram(n_qubits, p)

array([[-0.41562694+0.5720614j ,  0.28760624+0.64597419j],
       [-0.41562694-0.5720614j ,  0.70323318-0.07391279j]])

#### Results of test
* Looks like our matrix transform matches what is expected

### Controlled Gate Function
$\newcommand{\ket}[1]{\left|{#1}\right\rangle}$
* Here we want our single qubit transformation to end up in the upper right hand corner of the matrix.
* That is, we want to send $\ket{00} \to a\ket{00} + b\ket{01}$ and $\ket{01} \to c\ket{00} + d\ket{01}$ (and everything else to 0).
* This is just for ease of reading that we got the correct thing.
* We do this by making our target qubit qubit 0, and flipping the control qubit at the beginning, and again at the end.

In [10]:
# Define control and target qubit
c = 1
t = 0

# Flip control qubit
p = Program(X(c))

# Apply controlled gate to superposition
p.inst( llqca.controlGate([c], [t], Q) )

# Flip control qubit again
p.inst( Program(X(c)) )

# Get matrix corresponding to controlled gate
pd.DataFrame(qch.getMatFromProgram(2, p))

,0,1,2,3
0,(-0.4156269377774533+0.5720614028176844j),(0.2876062384759507+0.645974188021251j),0j,0j
1,(-0.4156269377774536-0.5720614028176843j),(0.703233176253404-0.07391278520356664j),0j,0j
2,0j,0j,(1+1.1102230246251565e-16j),(-1.6314318015755256e-17+2.2454732361702685e-17j)
3,0j,0j,(1.6314318015755256e-17+2.2454732361702685e-17j),(1-5.551115123125783e-17j)


#### Results of test
* Looks like we get the right matrix (upper right quadrant of the matrix corresponds to our originally predicted arbitrary matrix).

### Test k-controlled gate function
* Again we want the actual submatrix representing the arbitrary one-qubit transformation to be in the upper left hand corner.
* That is, we want to send $\ket{0...00} \to a\ket{0...00} + b\ket{0...01}$ and $\ket{0...01} \to c\ket{0...00} + d\ket{0...01}$ (and everything else to 0).
* To do this, the target qubit needs to be qubit 0, and the control bit sequence needs to be 00...0

In [11]:
# Define control bit sequence, with second array indicating whether to use Q or XQX
z = [[0, 0, 0], [0]]

# Define qubit registers
a = [1, 2, 3]
b = [0]
t = [[4], [5]]

p = Program()

# Apply k-qubit control gate to superposition
p.inst( llqca.kControlGate(z, Q, a, b, t) )

# Get matrix corresponding to controlled gate
pd.DataFrame(qch.getMatFromProgram(4, p))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,(-0.4156269377774533+0.5720614028176844j),(0.2876062384759507+0.645974188021251j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
1,(-0.4156269377774536-0.5720614028176843j),(0.703233176253404-0.07391278520356664j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
2,0j,0j,(1+1.1102230246251565e-16j),(-1.6314318015755256e-17+2.2454732361702685e-17j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
3,0j,0j,(1.6314318015755256e-17+2.2454732361702685e-17j),(1-5.551115123125783e-17j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
4,0j,0j,0j,0j,(1+1.1102230246251565e-16j),(-1.6314318015755256e-17+2.2454732361702685e-17j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
5,0j,0j,0j,0j,(1.6314318015755256e-17+2.2454732361702685e-17j),(1-5.551115123125783e-17j),0j,0j,0j,0j,0j,0j,0j,0j,0j,0j
6,0j,0j,0j,0j,0j,0j,(1+1.1102230246251565e-16j),(-1.6314318015755256e-17+2.2454732361702685e-17j),0j,0j,0j,0j,0j,0j,0j,0j
7,0j,0j,0j,0j,0j,0j,(1.6314318015755256e-17+2.2454732361702685e-17j),(1-5.551115123125783e-17j),0j,0j,0j,0j,0j,0j,0j,0j
8,0j,0j,0j,0j,0j,0j,0j,0j,(1+1.1102230246251565e-16j),(-1.6314318015755256e-17+2.2454732361702685e-17j),0j,0j,0j,0j,0j,0j
9,0j,0j,0j,0j,0j,0j,0j,0j,(1.6314318015755256e-17+2.2454732361702685e-17j),(1-5.551115123125783e-17j),0j,0j,0j,0j,0j,0j


#### Test results
* Again, the upper-left matrix matches our predicted one: Thus, our function works properly.

## Test getControlStrings function

In [12]:
n_qubits = 3
zero_string = 5
one_string = 4
print("binary representation of", str(zero_string), "is", np.binary_repr(zero_string, width=n_qubits))
print("binary representation of", str(one_string), "is", np.binary_repr(one_string, width=n_qubits))

binary representation of 5 is 101
binary representation of 4 is 100


In [13]:
a, b, z = hlqca.getControlStrings(zero_string, one_string, n_qubits)
print("control register is:", a)
print("target register is:", b)
print("control string is:", z)

control register is: [1, 2]
target register is: [0]
control string is: [[0, 1], [1]]


#### Result of Test
* We see above that the binary representations of 5 and 4 differ in one place.
* Additionally, that place is the 0 place, meaning the 1, 2 places are the control places.
* Additionally, the zero string has a 1 in the target place, so we should apply XQX.
* The control strings returned by `getControlStrings` reflects exactly these facts.

# Creating an arbitrary n-qubit transformation

### Generating $W_m$
* The scheme here is to iteratively take the two last basis states, cancel out their complex phases, then transfer all of the amplitude from the last basis state to the second to last basis state, effectively eliminating the last basis state.
* If we continue to do this, we'll end up with all of the amplitude in $x_m$
* To do this, we first need to extract all of the complex phases, and do a global phase shift so the last vector has no phase.
* Then we generate the amplitudes that each of the basis vectors will have _on the iteration when they are the last basis vector_.
* Then we need to get a list of binary numbers in the standard gray code ordering -- these are the basis vectors which will correspond to the input amplitudes.
* Then we write our program, which consists of a global phase shift, then several controlled phase shifts and rotations to cancel off all of the basis vector amplitudes as stated before.

In [14]:
m = 2
t = [[n_qubits], [n_qubits + 1]]

# Generate vector, put in standard ordering
gc_vector = qch.genRandomVector(2**n_qubits, zero_places=m)
P = hlqca.permutationMatrix(2**n_qubits)
vector = gc_vector@P

# Generate wavefunction from vector
p = create_arbitrary_state(vector)
result = WavefunctionSimulator().wavefunction(p)
print("The initial random state is:", result, end="\n\n")

# Show effect of W_m on wavefunction
Wm_result = hlqca.genWm(gc_vector, n_qubits, t)
p.inst( Wm_result )
result = WavefunctionSimulator().wavefunction(p)
print("The final random state is:", result)

The initial random state is: (0.4514310327-0.086380106j)|010> + (0.5572956673+0.0724831132j)|011> + (0.2702114719+0.6076360927j)|100> + (0.0185232001+0.0405647271j)|101> + (0.059310305-0.1040691649j)|110> + (0.0280763065+0.1164276099j)|111>

The final random state is: (1+0j)|0011>


#### Result of Test
* Our program has taken the original wavefunction and collapsed it to the lowest nonzero basis vector.
* This is how we want it to behave.
* Additionally, we have generated a matrix (in the graycode ordering) which does the same thing as the program.

### Generating and testing the inverse of $W_m$, $C_m$
* I've written a function which generates the inverse of $W_m$ (this is equivalent to doing all the transformations backwards and negating all of the angles).
* So first I create a program which puts the quantum simulator into the $x_m$ basis state.
* Then I run that program, and the program which implements $C_m$ on the quantum simulator.
* It looks like I generate the desired state.

In [15]:
m = 3
t = [[n_qubits], [n_qubits + 1]]

# Generate vector, put in standard ordering
gc_vector = qch.genRandomVector(2**n_qubits, zero_places=m)
P = hlqca.permutationMatrix(2**n_qubits)
vector = gc_vector@P

# Generate wavefunction from vector
p = create_arbitrary_state(vector)
result = WavefunctionSimulator().wavefunction(p)
print("The initial random state is:", result, end="\n\n")

# Show effect of W_m on wavefunction
Wm_result = hlqca.genWm(gc_vector, n_qubits, t)
p.inst( Wm_result )
result = WavefunctionSimulator().wavefunction(p)
print("The random state after W_m is:", result, end="\n\n")

# Show effect of C_m on single basis state
Cm_result = hlqca.genCm(gc_vector, n_qubits, t)
p.inst( Cm_result )
result = WavefunctionSimulator().wavefunction(p)
print("The final random state after C_m is:", result, end="\n\n")

# Show effect of W_m matrix
Wm_mat = hlqca.genWmMat(gc_vector, n_qubits)
print("The effect of the Wm matrix is:", (Wm_mat@gc_vector).round(10))

The initial random state is: (0.5457705724-0.1044317215j)|010> + (0.3266799555+0.7346191867j)|100> + (0.0223941572+0.0490418973j)|101> + (0.0717049045-0.1258174196j)|110> + (0.0339436608+0.1407585183j)|111>

The random state after W_m is: (1+0j)|0010>

The final random state after C_m is: (0.5457705724-0.1044317215j)|0010> + (0.3266799555+0.7346191867j)|0100> + (0.0223941572+0.0490418973j)|0101> + (0.0717049045-0.1258174196j)|0110> + (0.0339436608+0.1407585183j)|0111>

The effect of the Wm matrix is: [ 0.+0.j  0.+0.j  0.+0.j  1.+0.j -0.-0.j -0.-0.j  0.-0.j  0.+0.j]


## Iterating to generate matrix
* Now we need to iteratively generate $C_m$'s to create the transformation.
* During this process we'll also need to generate $U_m$ so that we can continue to iterate.
* Generating $U_m$ just consists of multiplying $W_m$ by $U_{m-1}$, so if we can get $W_m$ in matrix form, then we're golden.

### Generating Random Unitary Operation
* To do this we first create a random, square, complex-valued matrix.
* Then we use QR decomposition (there is a function from the linalg package in scipy).
* From this, we get a random unitary. 
* This is done in the function `genRandomUnitary` from the `QCHelper` package.

In [16]:
n_qubits = 3
rand_seed = 42

U = qch.genRandomUnitary(2**n_qubits, rand_seed=rand_seed)

# Make sure it's unitary by multiplying by the adjoint
U = np.matrix(U)
df = pd.DataFrame(U)
df.apply(np.round, decimals=3)

,0,1,2,3,4,5,6,7
0,(-0.068+0.346j),(-0.579-0.142j),(-0.159+0.017j),(-0.214-0.431j),(0.099-0.071j),-0.083j,(-0.259+0.259j),(0.323-0.055j)
1,(0.566+0.02j),(0.168-0.23j),(0.416+0.247j),(0.061-0.416j),(0.288+0.209j),(-0.014-0.106j),(0.093-0.067j),(0.004-0.192j)
2,(0.187-0.217j),(-0.151-0.166j),(-0.064+0.197j),(0.304+0.25j),(0.126-0.498j),(0.045+0.195j),(0.244-0.051j),(0.549-0.05j)
3,(0.326-0.279j),(-0.351+0.137j),(-0.057+0.141j),(0.01-0.018j),(-0.167+0.272j),(0.56+0.177j),(-0.106-0.046j),(-0.072+0.431j)
4,(-0.061-0.009j),(-0.466+0.214j),(0.391-0.05j),(0.406+0.172j),(0.089+0.304j),(-0.506+0.121j),(-0.031+0.025j),(-0.007+0.103j)
5,(0.096-0.063j),(0.018+0.29j),(-0.19+0.349j),(0.118-0.308j),(0.011-0.443j),(-0.338-0.239j),(-0.15-0.209j),(-0.248+0.375j)
6,(0.465-0.22j),(0.099+0.001j),(-0.375-0.429j),(0.184-0.03j),(-0.313+0.156j),(-0.273-0.222j),(-0.217+0.121j),(0.216-0.107j)
7,(0.029-0.078j),(0.103-0.061j),(0.177+0.073j),(0.296+0.108j),(0.057-0.267j),(0.174-0.006j),(-0.373+0.713j),(-0.292-0.071j)


* Make sure it's unitary (and not just random)

In [17]:
df = pd.DataFrame(U@U.getH())
df.apply(np.round, decimals=3)

,0,1,2,3,4,5,6,7
0,(1+0j),(-0-0j),(-0-0j),0j,(-0+0j),-0j,(-0-0j),0j
1,(-0+0j),(1+0j),-0j,(-0+0j),-0j,-0j,-0j,-0j
2,(-0+0j),0j,(1+0j),(-0-0j),0j,0j,0j,(-0+0j)
3,-0j,(-0-0j),(-0+0j),(1+0j),(-0+0j),-0j,(-0+0j),(-0+0j)
4,(-0-0j),0j,-0j,(-0-0j),(1+0j),(-0+0j),0j,0j
5,0j,0j,-0j,0j,(-0-0j),(1+0j),(-0-0j),(-0+0j)
6,(-0+0j),0j,-0j,0j,-0j,(-0+0j),(1+0j),0j
7,-0j,0j,(-0-0j),(-0-0j),0j,(-0-0j),-0j,(1+0j)


### Testing matrix generation on random unitary matrix
* Now that we have a random unitary $U$, we can test its decomposition into gates.
* Basically we'll want to generate a program which is a stack of $C_m$'s.
* In order to generate the next $C_m$, we'll need to get the previous $W_m$ and from that generate a new $U_m$.

In [18]:
t = [[n_qubits], [n_qubits + 1]]

U = np.array(U)
p = hlqca.genNQubitTransform(U, n_qubits, t)

df = pd.DataFrame(U)
df.apply(np.round, decimals=5)

,0,1,2,3,4,5,6,7
0,(-0.06811+0.346j),(-0.57904-0.14161j),(-0.159+0.01719j),(-0.21374-0.4306j),(0.09871-0.07123j),(0.00043-0.08285j),(-0.25905+0.25928j),(0.32341-0.05501j)
1,(0.56563+0.01963j),(0.16839-0.22968j),(0.41598+0.24706j),(0.06132-0.41588j),(0.2875+0.20865j),(-0.01358-0.10622j),(0.093-0.06741j),(0.0041-0.19211j)
2,(0.18686-0.21712j),(-0.15122-0.16641j),(-0.06407+0.19694j),(0.30405+0.24976j),(0.12588-0.49776j),(0.04464+0.19468j),(0.244-0.05098j),(0.54911-0.04997j)
3,(0.32602-0.27947j),(-0.35066+0.13724j),(-0.05682+0.14053j),(0.00991-0.018j),(-0.1669+0.27173j),(0.55962+0.1773j),(-0.10627-0.04604j),(-0.0724+0.43065j)
4,(-0.06063-0.00872j),(-0.46563+0.21436j),(0.39069-0.05046j),(0.40587+0.17195j),(0.08912+0.30417j),(-0.50648+0.12142j),(-0.03083+0.02451j),(-0.00654+0.10334j)
5,(0.09606-0.06304j),(0.01812+0.28963j),(-0.19007+0.34891j),(0.11829-0.30812j),(0.01087-0.44272j),(-0.33828-0.23929j),(-0.1498-0.2087j),(-0.24768+0.37502j)
6,(0.46534-0.22005j),(0.09925+0.0009j),(-0.37547-0.42882j),(0.18358-0.03015j),(-0.31319+0.15578j),(-0.27317-0.2217j),(-0.21656+0.12084j),(0.21579-0.10731j)
7,(0.02942-0.07795j),(0.10312-0.06107j),(0.17652+0.07347j),(0.29584+0.10763j),(0.05733-0.26734j),(0.17437-0.00585j),(-0.37299+0.71328j),(-0.29151-0.07051j)


In [19]:
New_U = qch.getMatFromProgram(n_qubits, p)
df = pd.DataFrame(New_U)
df.apply(np.round, decimals=5)

,0,1,2,3,4,5,6,7
0,(-0.06811+0.346j),(-0.57904-0.14161j),(-0.159+0.01719j),(-0.21374-0.4306j),(0.09871-0.07123j),(0.00043-0.08285j),(-0.25905+0.25928j),(0.32341-0.05501j)
1,(0.56563+0.01963j),(0.16839-0.22968j),(0.41598+0.24706j),(0.06132-0.41588j),(0.2875+0.20865j),(-0.01358-0.10622j),(0.093-0.06741j),(0.0041-0.19211j)
2,(0.18686-0.21712j),(-0.15122-0.16641j),(-0.06407+0.19694j),(0.30405+0.24976j),(0.12588-0.49776j),(0.04464+0.19468j),(0.244-0.05098j),(0.54911-0.04997j)
3,(0.32602-0.27947j),(-0.35066+0.13724j),(-0.05682+0.14053j),(0.00991-0.018j),(-0.1669+0.27173j),(0.55962+0.1773j),(-0.10627-0.04604j),(-0.0724+0.43065j)
4,(-0.06063-0.00872j),(-0.46563+0.21436j),(0.39069-0.05046j),(0.40587+0.17195j),(0.08912+0.30417j),(-0.50648+0.12142j),(-0.03083+0.02451j),(-0.00654+0.10334j)
5,(0.09606-0.06304j),(0.01812+0.28963j),(-0.19007+0.34891j),(0.11829-0.30812j),(0.01087-0.44272j),(-0.33828-0.23929j),(-0.1498-0.2087j),(-0.24768+0.37502j)
6,(0.46534-0.22005j),(0.09925+0.0009j),(-0.37547-0.42882j),(0.18358-0.03015j),(-0.31319+0.15578j),(-0.27317-0.2217j),(-0.21656+0.12084j),(0.21579-0.10731j)
7,(0.02942-0.07795j),(0.10312-0.06107j),(0.17652+0.07347j),(0.29584+0.10763j),(0.05733-0.26734j),(0.17437-0.00585j),(-0.37299+0.71328j),(-0.29151-0.07051j)


In [20]:
df = pd.DataFrame(U - New_U)
df.apply(np.round, decimals=5)

,0,1,2,3,4,5,6,7
0,-0j,0j,(-0-0j),0j,(-0+0j),0j,-0j,(-0+0j)
1,(-0+0j),(-0+0j),(-0+0j),(-0+0j),(-0+0j),(-0+0j),(-0-0j),-0j
2,-0j,(-0+0j),(-0+0j),(-0-0j),(-0+0j),(-0-0j),(-0-0j),(-0+0j)
3,(-0+0j),-0j,-0j,0j,-0j,0j,0j,(-0-0j)
4,-0j,0j,(-0-0j),(-0-0j),-0j,0j,(-0+0j),-0j
5,(-0-0j),(-0-0j),-0j,(-0+0j),(-0+0j),0j,-0j,-0j
6,(-0+0j),-0j,0j,(-0-0j),-0j,(-0+0j),-0j,(-0+0j)
7,(-0+0j),-0j,(-0+0j),(-0-0j),0j,0j,-0j,-0j


# Quantum Fourier Transform

### Iterative Phase Estimation Algorithm
* Here we try to get the phase estimation from a small number of qubits.
* Hopefully we can do this by applying $(2\omega)$ and then $(4\omega)$ and so on.

$\newcommand{\ket}[1]{\left|{#1}\right\rangle}
\newcommand{\bra}[1]{\left\langle{#1}\right|}$
#### First, we create the following superposition:
$$
    \ket{\psi} = \frac{1}{\sqrt{n}} \left( \ket{000...} + e^{-2\pi f}\ket{00...01} + e^{-2\pi 2f}\ket{00...10} + ... + e^{-2\pi nf}\ket{111...} \right)
$$

In [21]:
n_qubits = 4
f = 1/4
omega = -2*np.pi*f
M = n_qubits**2
n = np.array([i for i in range(M)])
x = np.exp(omega*1j*n)

p = create_arbitrary_state(x)
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)

(0.25+0j)|0000> + -0.25j|0001> + (-0.25+0j)|0010> + 0.25j|0011> + (0.25+0j)|0100> + -0.25j|0101> + (-0.25+0j)|0110> + 0.25j|0111> + (0.25+0j)|1000> + -0.25j|1001> + (-0.25+0j)|1010> + 0.25j|1011> + (0.25+0j)|1100> + (-0-0.25j)|1101> + (-0.25+0j)|1110> + 0.25j|1111>


* Then we run the quantum fourier transform. 
* Based on the continuous transform, the basis state corresponding to the value of f should have the highest amplitude.

In [22]:
p = create_arbitrary_state(x)
p.inst( qft([i for i in range(n_qubits)]) )
wfn = WavefunctionSimulator().wavefunction(p)

print(wfn, end="\n\n")
print(np.argmax(np.abs(wfn.amplitudes)))
print(np.argmax(np.abs(wfn.amplitudes))/len(wfn.amplitudes))

(1+0j)|0100>

4
0.25


* We notice that the Fourier transformation of the wavefunction is exactly the binary representation of the phase.
* i.e. the binary representation of 1/4 is 0.0100 and the state with the highest amplitude is 0100
* I guess we will be able to measure the phase to 1/2^n with n qubits.
* We can try again with a different phase.

In [23]:
n_qubits = 4
f = 1/8
omega = -2*np.pi*f
M = n_qubits**2
n = np.array([i for i in range(M)])
x = np.exp(omega*1j*n)

p = create_arbitrary_state(x)
p.inst( qft([i for i in range(n_qubits)]) )
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn, end="\n\n")
print(np.argmax(np.abs(wfn.amplitudes)))
print(np.argmax(np.abs(wfn.amplitudes))/len(wfn.amplitudes))

(1+0j)|0010>

2
0.125


### Higher decimal expansion
* Now we can try to do something that has a higher decimal expansion.
* Try $f = \frac{1}{13} \approx \frac{1}{16} + \frac{1}{128} + \frac{1}{256} + \frac{1}{512} + \frac{1}{2048} + \frac{1}{4096}$
* This means that the binary representation of $\frac{1}{13} = 0.000100111011...$
* In this scenario I am just going to print out the binary representation of the index with the larges amplitude

In [24]:
n_qubits = 4
n_digits = 7
f = 1/13

for i in range(n_digits):

    # Make vector representing wavefunction after
    # it has been set up by phase estimation alg.
    omega = 2*np.pi*(f)
    M = n_qubits**2
    n = np.array([i for i in range(M)])
    x = np.exp(-omega*1j*n)

    # Encode the generation of that state into a q-program
    p = create_arbitrary_state(x)
    
    # Perform quantum fourier transform, run program
    p.inst( qft([i for i in range(n_qubits)]) )
    wfn = WavefunctionSimulator().wavefunction(p)

    # Find basis state with highest amplitude - get binary rep
    max_idx = np.argmax(np.abs(wfn.amplitudes))
    bin_phase = np.binary_repr(max_idx, width=n_qubits)
    
    print("f is", f)
    print("In binary, f is", qch.getBinFrac(f, n_qubits))
    print("State with highest magnitude is:", bin_phase)
    
    # If the first digit is 1 in binary representation,
    # subtract it off (don't want f to go over 1)
    if int(bin_phase[0]):
        f -= 1/2
        print("Was true")
      
    # Multiply everything by 2 (shifts binary representation
    # over by one bit)
    f *= 2
    
    print()

f is 0.07692307692307693
In binary, f is 0.0001
State with highest magnitude is: 0001

f is 0.15384615384615385
In binary, f is 0.0010
State with highest magnitude is: 0010

f is 0.3076923076923077
In binary, f is 0.0100
State with highest magnitude is: 0101

f is 0.6153846153846154
In binary, f is 0.1001
State with highest magnitude is: 1010
Was true

f is 0.23076923076923084
In binary, f is 0.0011
State with highest magnitude is: 0100

f is 0.4615384615384617
In binary, f is 0.0111
State with highest magnitude is: 0111

f is 0.9230769230769234
In binary, f is 0.1110
State with highest magnitude is: 1111
Was true



* It looks like, although the phase estimation algorithm wasn't accurate to all bit places all the time, the first bit after each successive iteration was always correct.

### Writing Stock Phase Estimation Algorithm
* Now that we know how the iterated phase estimation algorithm will work, I want to write a method which takes in a program, and does the phase estimation algorithm.
* I will begin by trying to estimate the phase from the phase gate.
* For this I need to apply the phase gate to the nth basis state n times.
     1. To do this, we first flip an auxiliary qubit to 1 for all the states.
     2. Then, we instantiate a for loop to run for n times.
     3. In the for loop, we first flip the auxiliary qubit in the nth basis state (this can be done with the AND operation).
     4. Then we apply the gate which will be controlled by the auxiliary qubit.
     
     
* By doing this, we will have applied the gate n times to the nth basis.
* Then we just have to apply the quantum fourier transform to the qubits used in the enumeration.

#### The following code does the PEA superposition (right before qft)

In [25]:
n_qubits = 4
a = [i for i in range(n_qubits)]
b = [n_qubits + 1]
t = [n_qubits + 2]
M = 2**n_qubits
f = 1/13
phase = -2*np.pi*f
Q = {"delta": phase}

p = Program()

# Create equal superposition
for i in range(n_qubits):
    p.inst(Program(H(i)))
    
# Flip auxiliary qubit
p.inst( Program(X(b[0])) )
    
# Apply phase gate to the ith basis vector i times
for i in range(M):
    # Get control string for current basis vector
    ctrl_str = np.binary_repr(i, width=n_qubits)
    ctrl_str = [int(j) for j in ctrl_str]
    ctrl_str.reverse()
    
    # Negate control qubit in ith basis vector
    for j in range(len(ctrl_str)):
        if not ctrl_str[j]:
            p.inst(X(j))
            
    p.inst( llqca.And(a, b, t) )
    
    for j in range(len(ctrl_str)):
        if not ctrl_str[j]:
            p.inst( X(j) )
            
    # Apply gate controlled by auxiliary qubit
    p.inst( llqca.controlGate([n_qubits + 1], [0], Q) )
    
# Print out the relevant amplitudes
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn.amplitudes[:M])
print()

# Generate amplitudes manually
n = np.array([i for i in range(M)])
test_phase = np.exp(1j*n*phase)
test_phase = ( 1/np.linalg.norm(test_phase) )*test_phase
print(test_phase)

[ 0.25      +4.85722573e-17j  0.22136401-1.16180793e-01j
  0.14201619-2.05745966e-01j  0.03013417-2.48177219e-01j
 -0.08865122-2.33754061e-01j -0.18712769-1.65780665e-01j
 -0.24273545-5.98289161e-02j -0.24273545+5.98289161e-02j
 -0.18712769+1.65780665e-01j -0.08865122+2.33754061e-01j
  0.03013417+2.48177219e-01j  0.14201619+2.05745966e-01j
  0.22136401+1.16180793e-01j  0.25      -1.94289029e-16j
  0.22136401-1.16180793e-01j  0.14201619-2.05745966e-01j]

[ 0.25      +0.00000000e+00j  0.22136401-1.16180793e-01j
  0.14201619-2.05745966e-01j  0.03013417-2.48177219e-01j
 -0.08865122-2.33754061e-01j -0.18712769-1.65780665e-01j
 -0.24273545-5.98289161e-02j -0.24273545+5.98289161e-02j
 -0.18712769+1.65780665e-01j -0.08865122+2.33754061e-01j
  0.03013417+2.48177219e-01j  0.14201619+2.05745966e-01j
  0.22136401+1.16180793e-01j  0.25      +6.12323400e-17j
  0.22136401-1.16180793e-01j  0.14201619-2.05745966e-01j]


* Looks like we have agreement between the phases generated by our function and by the phases generated manually.
* Now we have to apply the quantum fourier transform.

In [26]:
# Perform QFT
p.inst( qft([i for i in range(n_qubits)]) )
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)
print()

# Print out the first n_qubits
max_idx = np.argmax(np.abs(wfn.amplitudes))
print("State with the highest magnitude is:", np.binary_repr(max_idx, width=n_qubits))

(0.1533450483-0.0804816899j)|000000> + (0.7116575508-0.575103406j)|000001> + (-0.1763454301+0.2115985868j)|000010> + (-0.0581965501+0.1069278128j)|000011> + (-0.0238336792+0.0764849656j)|000100> + (-0.0064857834+0.061116071j)|000101> + (0.0046566958+0.0512446958j)|000110> + (0.0129539301+0.0438939877j)|000111> + (0.0198369549+0.0377961598j)|001000> + (0.0260766078+0.0322683097j)|001001> + (0.0322047136+0.0268392817j)|001010> + (0.0387154465+0.0210712757j)|001011> + (0.0462367736+0.0144079613j)|001100> + (0.0558141655+0.0059231325j)|001101> + (0.0696435489-0.0063286321j)|001110> + (0.0937200069-0.0276585126j)|001111>

State with the highest magnitude is: 0001


In [27]:
qch.getBinFrac(f, n_qubits)

'0.0001'

* Good news, it looks like we get the right reading
* Now we need to generalize and package everything into the right function

### Testing library function: `initializePEA`

In [28]:
# Setup inputs to function
n_qubits = 4
a = [i for i in range(n_qubits)]
c = [n_qubits + 1]
t = [n_qubits + 2]
M = 2**n_qubits
f = 1/13
phase = -2*np.pi*f
Q = {"delta": phase}
U = llqca.controlGate(c, [0], Q)

# Run function
p = hlqca.initializePEA(U, a, c, t)

# Print out the relevant amplitudes
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn.amplitudes[:2**n_qubits])
print()

# Generate amplitudes manually
n = np.array([i for i in range(2**n_qubits)])
test_phase = np.exp(1j*n*phase)
test_phase = ( 1/np.linalg.norm(test_phase) )*test_phase
print(test_phase)
print()

# Print out difference between amplitudes
diff = (test_phase - wfn.amplitudes[:2**n_qubits])
diff = np.around(diff, decimals=7)
print(diff)

[ 0.25      +4.85722573e-17j  0.22136401-1.16180793e-01j
  0.14201619-2.05745966e-01j  0.03013417-2.48177219e-01j
 -0.08865122-2.33754061e-01j -0.18712769-1.65780665e-01j
 -0.24273545-5.98289161e-02j -0.24273545+5.98289161e-02j
 -0.18712769+1.65780665e-01j -0.08865122+2.33754061e-01j
  0.03013417+2.48177219e-01j  0.14201619+2.05745966e-01j
  0.22136401+1.16180793e-01j  0.25      -1.94289029e-16j
  0.22136401-1.16180793e-01j  0.14201619-2.05745966e-01j]

[ 0.25      +0.00000000e+00j  0.22136401-1.16180793e-01j
  0.14201619-2.05745966e-01j  0.03013417-2.48177219e-01j
 -0.08865122-2.33754061e-01j -0.18712769-1.65780665e-01j
 -0.24273545-5.98289161e-02j -0.24273545+5.98289161e-02j
 -0.18712769+1.65780665e-01j -0.08865122+2.33754061e-01j
  0.03013417+2.48177219e-01j  0.14201619+2.05745966e-01j
  0.22136401+1.16180793e-01j  0.25      +6.12323400e-17j
  0.22136401-1.16180793e-01j  0.14201619-2.05745966e-01j]

[ 0.-0.j  0.-0.j  0.-0.j  0.-0.j  0.-0.j  0.-0.j -0.-0.j -0.-0.j -0.-0.j
 -0.-0.j -0

* Good, it looks like our manual generation of the inital superposition gives the same answer as when we do it using our quantum algorithm.

### See if we can accurately find phase with `initializePEA`

In [70]:
# Setup inputs to function
n_qubits = 4
a = [i + 1 for i in range(n_qubits)]
c = [n_qubits + 1]
t = [n_qubits + 2]
M = 2**n_qubits
f = 1/13
phase = -2*np.pi*f
Q = {"delta": phase}
U = llqca.controlGate(c, [0], Q)

# Run function
p = hlqca.initializePEA(U, a, c, t)

# Perform QFT
p.inst( qft([i + 1 for i in range(n_qubits)]) )
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)
print()

# Print out the first n_qubits
max_idx = np.argmax(np.abs(wfn.amplitudes))
print("State with the highest magnitude is:", np.binary_repr(max_idx, width=7))

(0.1533450483-0.0804816899j)|0000000> + (0.7116575508-0.575103406j)|0000010> + (-0.1763454301+0.2115985868j)|0000100> + (-0.0581965501+0.1069278128j)|0000110> + (-0.0238336792+0.0764849656j)|0001000> + (-0.0064857834+0.061116071j)|0001010> + (0.0046566958+0.0512446958j)|0001100> + (0.0129539301+0.0438939877j)|0001110> + (0.0198369549+0.0377961598j)|0010000> + (0.0260766078+0.0322683097j)|0010010> + (0.0322047136+0.0268392817j)|0010100> + (0.0387154465+0.0210712757j)|0010110> + (0.0462367736+0.0144079613j)|0011000> + (0.0558141655+0.0059231325j)|0011010> + (0.0696435489-0.0063286321j)|0011100> + (0.0937200069-0.0276585126j)|0011110>

State with the highest magnitude is: 0000010


In [69]:
qch.getBinFrac(f, n_qubits)

'0.0001'

* Success! Note that our register qubits (the ones that tell us what the phase is) are in qubit places 0, 2, 3,  and 4, so we have to read those off when comparing to the binary representation of our frequency

### Testing out measurement

 * One way to measure is by using the `run_and_measure()` function.
 * For this, you need a complete program.
 * The result of this function is to give you back a dictionary of arrays, where each qubit location is held as an array whose $i$th value is the value of that qubit on the $i$th measurement. 

In [34]:
qubit = 0
p = Program(X(qubit))
qc = get_qc('5q-qvm')

bitstrings = qc.run_and_measure(p, trials=10)
bitstrings

{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

In [35]:
p = hlqca.initializePEA(U, a, t)
p += qft(a)

qc = get_qc('7q-qvm')

bitstrings = qc.run_and_measure(p, trials=10)
bitstrings

{0: array([1, 0, 1, 1, 1, 1, 1, 1, 1, 1]),
 1: array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 4: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 5: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 6: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

 * Additionally, we can introduce measurement like a gate
 * To do this, we first need to allocate classical memory.
 * Then, we use a MEASURE gate to measure the qubit and put the corresponding measurement into classical memory.

In [36]:
program = Program()
ro = program.declare('ro', memory_type='BIT', memory_size=2)
program += H(0)
program += CNOT(0, 1)
program += MEASURE(0, ro[0])
program += MEASURE(1, ro[1])
print(program)

DECLARE ro BIT[2]
H 0
CNOT 0 1
MEASURE 0 ro[0]
MEASURE 1 ro[1]



In [37]:
for i in range(10):
    print(qc.run(program))

[[0 0]]
[[1 1]]
[[1 1]]
[[0 0]]
[[0 0]]
[[0 0]]
[[0 0]]
[[0 0]]
[[0 0]]
[[0 0]]


 * Note that you cannot get the resulting wavefunction after you have measured
 * I'm working on trying to get this to see if the wavefunctions collapse to the eigenvectors when we measure the phase to be the eigenvalues.

# Testing Phase Estimation Algorithms

### We will complete this task as follows:
1. Make Random Unitary `U`
2. Find eigenvalues/eigenvectors
3. Choose an eigenvector and normalize it
4. Encode eigenvector into quantum state
5. Run phase estimation algorithm using `U` as gate
6. See with what proportion we get the correct phase

### Find eigenvalues/eigenvectors

In [4]:
seed = 41
n_qubits = 1
k = 2**n_qubits

# Find eigenvectors/eigenvalues
U = qch.genRandomUnitary(k, rand_seed=seed)
w, v = np.linalg.eig(U)

# Make sure they are correct
print( r'Uv - wv given by:', end='\n')
for i in range(len(w)):
    print((np.dot(U, v[:, i]) - w[i]*v[:, i]).round(10))

Uv - wv given by:
[-0.-0.j -0.-0.j]
[-0.+0.j -0.+0.j]


### Choose eigenvector with lowest phase (ground state)

In [5]:
# Get phase angles as between 0 and 2*pi
angles = np.abs(np.angle(w) - 2*np.pi)
angles = angles % (2*np.pi)
angles = -angles

# Find where minimum eigenvector is
min_place = np.argmin(np.abs(angles))

In [6]:
min_place = 1

### Encode ground state eigenvector into quantum computer

In [7]:
p = create_arbitrary_state(v[:, min_place])

### Make controlled unitary gate

In [21]:
cU = np.eye(2*k, dtype=np.complex_)
cU[k:, k:] = U
pd.DataFrame(cU)

,0,1,2,3
0,(1+0j),0j,0j,0j
1,0j,(1+0j),0j,0j
2,0j,0j,(-0.2586868266348876-0.23220857632306197j),(0.7256288913141814+0.5938206925476239j)
3,0j,0j,(-0.34006294769575424-0.8737948812004237j),(-0.14073663926629115-0.31785672174808843j)


* This seems right -- should send $\ket{10} \to \ket{1}U\ket{0}$ and $\ket{11} \to \ket{1}U\ket{1}$, and leave alone $\ket{00}$ and $\ket{01}$. 

In [37]:
n_qubits = 2
k = 2**n_qubits
U = qch.genRandomUnitary(k, rand_seed=seed)
pd.DataFrame(U)

,0,1,2,3
0,(-0.07174065688688702-0.2468320681669116j),(-0.1477469521304042+0.1356961861184421j),(0.08990577351192008+0.7105604186011794j),(-0.18651078300970686-0.5881489290227553j)
1,(-0.09273630297210536-0.07499332466215142j),(-0.14276164722506038-0.7933067602930793j),(0.4675950155873379+0.02674919383251491j),(0.3254613148420397-0.10379686936775741j)
2,(0.1047602321863669+0.933977136166032j),(-0.07271142122596802+0.01897010018930813j),(0.08426341984165614+0.3038002228500424j),(0.10232423550993908-0.03464157752792835j)
3,(0.15436069356170307+0.11219610864719114j),(0.01261332069345078-0.5515749885801228j),(-0.409770473271461-0.018801205851324845j),(-0.6980586211474948-0.060328939461719744j)


In [45]:
reload(hlqca)

<module 'HighLevelQCAlgorithms' from 'C:\\Users\\Lucas\\Documents\\Senior Experience\\Simulation Code\\HighLevelQCAlgorithms.py'>

In [53]:
n_qubits = 2
t = [[n_qubits], [n_qubits + 1]]
U = np.eye(2**n_qubits)

P = hlqca.permutationMatrix(2**n_qubits)
gray_code_U = P@U@P.T

m = 0
v = gray_code_U[:, m]

trimmed_v, nonzero_places = hlqca.parseVector(v, n_qubits)
gc_basis = hlqca.getGraycodeBasis(n_qubits, nonzero_places)
theta, phi = hlqca.calcAmplitudesPhases(trimmed_v)

p = Program()

# Change global phase so that last basis vector has no phase
# a, b, z = getControlStrings(gc_basis[-2], gc_basis[-1], n_qubits)
# Q = {'delta': phi[-1]/2, 'alpha': -phi[-1]/2}
# p.inst( llqca.kControlGate(z, Q, a, b, t) )

In [74]:
help(hlqca.getControlStrings)

Help on function getControlStrings in module HighLevelQCAlgorithms:

getControlStrings(zero_string, one_string, n_qubits)
    Return list of indices representing the control register, the target
    register, and the binary control string (with the bit denoting Q vs. XQX),
    given the "zero string" and the "one string". 
    
    Here the "zero string" is the string representing the basis vector which
    the gate will treat as 0, and the "one string" it will treat as 1. All 
    other basis vectors will remain unchanged. The "zero string" and the
    "one string" should differ by exactly one qubit, and that qubit will be
    the target qubit. If the "zero string" has a 0 in the target qubit place,
    a k-controlled gate will be applied. If it has a 1 in the target qubit
    place, a k-controlled gate XQX will be applied.
    
    Parameters
    ----------
    zero_string: integer
        An integer whose binary representation is the "zero string".
    one_string: integer
        An

In [65]:
new_v = np.zeros(v.shape, dtype=np.complex_)

In [69]:
new_v[0] = 1j
trimmed_v, nonzero_places = hlqca.parseVector(new_v, n_qubits)
gc_basis = hlqca.getGraycodeBasis(n_qubits, nonzero_places)
theta, phi = hlqca.calcAmplitudesPhases(trimmed_v)

p = Program()

In [81]:
nonzero_places[0] == 0

True

In [51]:
print(v)

[1. 0. 0. 0.]


In [46]:
n_qubits = 2
c = [1]
t = [[n_qubits], [n_qubits + 1]]

U_gate = hlqca.genNQubitTransform(np.eye(2**n_qubits), n_qubits, t)

0


IndexError: index -2 is out of bounds for axis 0 with size 1

### Run phase estimation algorithm on $U$

In [120]:
c = [4]
a = [5, 6, 7, 8]
t = [[9], [10]]

U_gate = hlqca.genNQubitTransform(U, n_qubits, t)
p += hlqca.initializePEA(U_gate, a, c, t[0])
p += qft(a)
wfn = WavefunctionSimulator().wavefunction(p)

# Print out the relevant amplitudes
wfn = WavefunctionSimulator().wavefunction(p)
print(wfn)
print()
max_idx = np.argmax(np.abs(wfn.amplitudes))
print("State with the highest magnitude is:", np.binary_repr(max_idx, width=9))

Exception: c does not have appropriate length

In [113]:
qch.getBinFrac(-angles[min_place]/(2*np.pi), 4)

'0.1101'

In [112]:
angles[min_place]/(2*np.pi)

-0.8703221261646437